# script questions to LLM and Server

In [5]:
import random
import json

In [1]:
from datasets import load_dataset
import requests
import time
import re
import matplotlib.pyplot as plt
mmlu_ds = load_dataset("cais/mmlu", "all")
df = mmlu_ds["validation"].to_pandas()

/home/tamar/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_num_to_letter(num):
    mapping = {0: 'a', 1: 'b', 2: 'c', 3: 'd'}
    return mapping.get(num, None)


In [3]:
response_times = []
correct_Server = []
correct_llm = []

In [4]:
import re

def extract_final_answer(model_response: str,kind:str) -> str | None:
    print(f"{kind}:\n{model_response}\n")
    match = re.search(r'Final answer:\s*([a-dA-D])', model_response)
    if match:
        return match.group(1).lower()
    return None


In [6]:
def ask_local_llm(question: str):
    url = "http://127.0.0.1:8013/generate"
    headers = {"Content-Type": "application/json"}
    payload = {
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question}
                ]
            }
        ]
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  
        return response.json()
    except requests.RequestException as e:
        print(f"❌ שגיאה בשליחה לשרת: {e}")
        return None

In [7]:
def send_message_to_backend( request_text: str):

    url = "http://localhost:8002/api/message/add"
    headers = {"Content-Type": "application/json"}
    payload = {
        "userId": "demo_user",
        "request": request_text
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()  # מחזירה את התגובה כ־dict
    except requests.RequestException as e:
        print(f"❌ שגיאה בשליחה לשרת הראשי: {e}")
        return None